In [1]:
import pandas as pd

from thresh import load_interface
import git, os, shutil
from github import Github

In [2]:
df = pd.read_csv('quarel_with_rationales.csv')
df.head()

,id,text,label,rationale
0,QuaRel_V1_Fr_0223,Question: Mike was snowboarding on the snow an...,B,The answer is B because the question states th...
1,QuaRel_V1_B5_1403,Question: John and Rita are going for a run. ...,A,"The answer is A, because the question does not..."
2,QuaRel_V1_Fr_0455,Question: The propeller on Kate's boat moved s...,B,The answer is B because the question states th...
3,QuaRel_V1_Fr_0334,Question: A car gets very hot as it drives up ...,A,The answer is A. The question states that the ...
4,QuaRel_V1_B5_1282,"Question: Juan is injured in a car accident, w...",B,The answer is B because the question states th...


In [3]:
df.iloc[0].text

'Question: Mike was snowboarding on the snow and hit a piece of ice. He went much faster on the ice because _____ is smoother. (A) snow (B) ice\n'

In [4]:
df.rename(columns={'text': 'context' , 'label':'source', 'rationale': 'target'}, inplace=True)

In [5]:
df.head()

,id,context,source,target
0,QuaRel_V1_Fr_0223,Question: Mike was snowboarding on the snow an...,B,The answer is B because the question states th...
1,QuaRel_V1_B5_1403,Question: John and Rita are going for a run. ...,A,"The answer is A, because the question does not..."
2,QuaRel_V1_Fr_0455,Question: The propeller on Kate's boat moved s...,B,The answer is B because the question states th...
3,QuaRel_V1_Fr_0334,Question: A car gets very hot as it drives up ...,A,The answer is A. The question states that the ...
4,QuaRel_V1_B5_1282,"Question: Juan is injured in a car accident, w...",B,The answer is B because the question states th...


In [6]:
df.sample(10).to_json("quarel.json", orient='records')

In [7]:
interface = load_interface('interface.yml')

In [8]:
local_template_path = "interface.yml"
local_data_path = "quarel.json"
repository_name = "prolific-deployment"

In [9]:
if os.path.exists('.git_credentials'):
    # This will read the API token from a local 
    github_username = "datalabNU"
    github_token = open('.git_credentials', 'r').read()
else:
    github_username = input("Please enter your GitHub username: ")
    github_token = getpass("Please enter your GitHub access token (register a token at github.com/settings/tokens): ")

# Log into GitHub
g = Github(github_token)
user = g.get_user()

In [10]:
try:
    remote_repo = g.get_repo(f"{github_username}/{repository_name}").clone_url
    print(f"Found existing repository: {remote_repo}")
except Exception as e:
    remote_repo = user.create_repo(repository_name).clone_url
    print(f"Created new repository: {remote_repo}")

if not os.path.exists(repository_name):
    git.Repo.clone_from(remote_repo, repository_name)

repo = git.Repo(repository_name)


Found existing repository: https://github.com/datalabNU/prolific-deployment.git


In [16]:
for file_path in [local_template_path, local_data_path]:
    template_repo_path = os.path.join(os.getcwd(), repository_name, file_path)
    shutil.copy(file_path, template_repo_path)
    repo.index.add([template_repo_path])
repo.index.commit("Add a new interface template and data")

origin = repo.remote("origin")

In [17]:
origin.push()
print(f"Pushed changes to {remote_repo}")

Pushed changes to https://github.com/datalabNU/prolific-deployment.git


In [18]:
template_path = f"{github_username}/{repository_name}/master/{local_template_path}"
data_path = f"{github_username}/{repository_name}/master/{local_data_path}"

print(f"Hosted template: {template_path}")
print(f"Hosted data: {data_path}")

Hosted template: datalabNU/prolific-deployment/master/interface.yml
Hosted data: datalabNU/prolific-deployment/master/quarel.json


In [19]:
study_url = f"https://thresh.tools/?gh={template_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

In [20]:
print(f"View your hosted template here:\t\t  {study_url}")

study_url = f"{study_url}&d={data_path}"

print(f"Your study (with data) will be linked to: {study_url}")

View your hosted template here:		  https://thresh.tools/?gh=datalabNU/prolific-deployment/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}
Your study (with data) will be linked to: https://thresh.tools/?gh=datalabNU/prolific-deployment/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&d=datalabNU/prolific-deployment/master/quarel.json
